In [ ]:
!pip install FastAPI  uvicorn Jinja2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken '2k3MeYFfEug55hrhwceISQC4we6_6aukuEMimUCi4K4tmS8dU'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install python-multipart


In [ ]:
!pip install apscheduler


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 1.0 MB/s eta 0:00:00


In [ ]:
app_code = """
import os
import base64
import asyncio
import uuid
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO, StringIO
from fastapi import FastAPI, Request, Form, File, UploadFile
from fastapi.templating import Jinja2Templates
from fastapi.responses import HTMLResponse, JSONResponse
from pydantic import BaseModel
from typing import List
import warnings

from predict import predict
from pipeline import preprocess_data_pipeline

warnings.simplefilter(action='ignore', category=FutureWarning)

from scheduler import schedule_task, tasks, TaskStatus, global_store

app = FastAPI()
templates = Jinja2Templates(directory="templates")

#Endpoint for initial form
@app.get("/", response_class=HTMLResponse)
async def get_form(request: Request):
    print("GET / - Showing form")
    return templates.TemplateResponse("home.html", {"request": request})

#Endpoint pour gerer une taches (via le planificateur)
@app.post("/", response_class=HTMLResponse)
async def handle_form(request: Request, name: str = Form(...), file: UploadFile = File(...), scale: str = Form(...)):
    print("POST / - Handling form submission")
    print("TASKS QUEUE BEFORE NEW TASK:")
    print(tasks)
    print(f"Number of tasks in queue: {len(tasks)}")

    task_id = str(uuid.uuid4())
    print(f"NEW TASK: {task_id}")

    tasks[task_id] = {"status": TaskStatus.WAITING}
    print("TASKS QUEUE AFTER NEW TASK:")
    print(tasks)

    f = await file.read()
    schedule_task(task_id, name, file, f, scale)

    print("TASKS QUEUE AFTER SCHEDULING TASK:")
    print(tasks)

    return templates.TemplateResponse("waiting.html", {"request": request, "task_id": task_id})

#Endpoint qui retourne l'etat
@app.get("/status/{task_id}", response_class=JSONResponse)
async def get_status(task_id: str):
    print(f"GET /status/{task_id} - Checking status")

    task = tasks.get(task_id)
    if not task:
        print(f"TASK {task_id} NOT FOUND")
        return JSONResponse(content={"error": "Task not found"}, status_code=404)

    # Ensure that only JSON serializable parts are included in the response
    task_status = {k: v for k, v in task.items() if k != 'result'}
    if 'result' in task:
        task_status['result'] = {rk: rv for rk, rv in task['result'].items() if rk != 'request'}

    print(f"STATUS OF TASK {task_id}: {task_status}")
    return JSONResponse(content=task_status)

class PredictionRequest(BaseModel):
    dates: List[str]

#Endpoint pour dredire
@app.post("/predict", response_class=JSONResponse)
async def handle_predict(request: Request, prediction_request: PredictionRequest):
    print("POST /predict - Handling prediction request")
    try:
        start_date = prediction_request.dates[0]
        end_date = prediction_request.dates[1]
        date_range = pd.date_range(start=start_date, end=end_date)
        dates = date_range.strftime('%Y-%m-%d').tolist()

        model = global_store.get("model")
        scaler_path = global_store.get("scaler_path")
        df_preprocessed = global_store.get("df_preprocessed")
        feature_names = global_store.get("feature_names")
        Y_variable = global_store.get("Y_variable")

        if model is None or df_preprocessed.empty:
            print("Prediction data not available")
            return JSONResponse(content={"error": "Prediction data not available"}, status_code=400)

        prediction_results = predict(model, scaler_path, dates, df_preprocessed, feature_names, Y_variable)
        prediction_html = prediction_results.to_html(index=False)

        print("Prediction results generated")
        return f"<h3>Prediction Results:</h3>{prediction_html}"
    except Exception as e:
        print(f"Error during prediction: {e}")
        return {"error": str(e)}

#Endpoint pour afficher le resultat d'une tache completee
@app.get("/results/{task_id}", response_class=HTMLResponse)
async def get_results(request: Request, task_id: str):
    print(f"GET /results/{task_id} - Fetching results")

    task = tasks.get(task_id)
    if not task or task['status'] != TaskStatus.DONE:
        print(f"TASK {task_id} NOT COMPLETED OR NOT FOUND")
        return templates.TemplateResponse("error.html", {"request": request, "error": "Task not completed yet or not found"})

    context = task['result']
    context["request"] = request

    print(f"RESULTS FOR TASK {task_id}: {context}")
    return templates.TemplateResponse("result.html", context)

if __name__ == "__main__":
    import uvicorn
    print("Starting FastAPI app")
    uvicorn.run(app, host="127.0.0.1", port=8000)


"""

with open("app.py", "w") as f:
    f.write(app_code)


In [ ]:
import uvicorn
from pyngrok import ngrok

ngrok.kill()
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

!uvicorn app:app --host 0.0.0.0 --port 8000 --reload

Public URL: NgrokTunnel: "https://aa1a-34-44-217-223.ngrok-free.app" -> "http://localhost:8000"
INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [19428] using StatReload
INFO:     Started server process [19430]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
GET / - Showing form
INFO:     154.108.156.198:0 - "GET / HTTP/1.1" 200 OK
INFO:     154.108.156.198:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
GET / - Showing form
INFO:     154.108.156.198:0 - "GET / HTTP/1.1" 200 OK
POST / - Handling form submission
TASKS QUEUE BEFORE NEW TASK:
{}
Number of tasks in queue: 0
NEW TASK: e5c1a5ba-95ee-4dbe-90f5-13a9140e3269
TASKS QUEUE AFTER NEW TASK:
{'e5c1a5ba-95ee-4dbe-90f5-13a9140e3269': {'status': 'waiting'}}
TASKS QUEUE AFTER SCHEDULING TASK:
{'e5c1a5ba-95ee-4dbe-90f5-13a9140e3269': {'status': 'waiting'}}
INFO:     154.108.156.198:

In [ ]:
pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.32.1
aiohappyeyeballs                 2.3.4
aiohttp                          3.10.0
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.13
albumentations                   1.4.12
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
APScheduler                      3.10.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.18.0
asn1crypto                       1.5.1
astropy                          6.1.2
astropy-iers-data                0.2024.7.29.0.32.7
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                   